In [7]:
import requests
import bs4

In [8]:
url = 'https://kozigallas.gov.hu/publicsearch.aspx'

In [9]:
keywords = ['gyógypedagógus','tanár','tanító','óvodapedagógus','orvos', 'védőnő']

In [10]:
def send_request(keyword):
    try:
        response = requests.post(
            url=url,
            params={
                "p": "1",
            },
            headers={
                "Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
                "Cookie": "ASP.NET_SessionId=nialgtjlnsftsb551ou1dj55",
            },
            data={
                "__EVENTTARGET": "",
                "__EVENTARGUMENT": "",
                "__VSTATE": "H4sIAAAAAAAEAO29B2AcSZYlJi9tynt/SvVK1+B0oQiAYBMk2JBAEOzBiM3mkuwdaUcjKasqgcplVmVdZhZAzO2dvPfee++999577733ujudTif33/8/XGZkAWz2zkrayZ4hgKrIHz9+fB8/Iv7vX/M3/e1+zV/3F/z+v/9JtWzrqmxe5b9oXdT5y6ppn2TTt79Xfv37//6/za/56/6W07bc2fnd3nz5cvd3m7TL59VFsfxNf91k+9N7ew8O6H/7v+nst/m1zvHPr/Vr0b+/3q/1a9oPzv2/f5tf5xf8WLFc5vW8XZS/7hePi8VF2tTTzz5a5LMiu1sssou8udvW2bJZZXW+bMYXxflH6VUxa+effbS7u/NROs+Li3lLf+zf+yidVPUsrz/7aOeju0e/4Nf/yaIpJmU+nzES3N3v8mt8w738Lr/mxezX+nUI/K/DY/p17Mh+LerqW8fpeV7Os+YHy3/0Ly7/0b91+Y/+1ZfpZXZxnWbpT+dl84N/9G9N58XkH/2Lm7HiyKB+LQLy2/xav+DXJhx/3d91PL4boNnkWT2d//6TddtWSyA6+7V+bdexIHH+m+L9X+dN/q7NZ4CMkf/av+5v/XsV5S/Ms+m6zQ9X5pfmd/k15zOB4SboNwIGv84v+HWnZdY0v+6vs8iXaxrOzuMsndf5+WcfrRiVb1eLfJw1q3cfHb24LtpfWAnEtCpnWfn4bnYEsNTzr2Mh7HUgnPykvv8Lv1wvysOlQarMaTZ++gcC49cVGL9Rk5f5lAaksD76hcfauPyFmY4lfZvXeWOHNvu1fv2w//ud/l/lF0XT0juz71STRnF5SXN1/Y/+xT/I2qxYCAo/hvlUMLNf6zcOgX7eAfqyrs6L0lDmmTKAwbE0ZCrS88L8/tZQy0zhr0eYo89f+9f9NXdm6B9/0ETxzPxms9/st/6Nfo3f+CD9R//gss3Kf/QvXlbN7/j6B9nbxXr5lpjp7T/6V69+8I//yWmxbP/Rv/oHefvb/17/6N/xA/rsvHr7j/49Kb7Nm3/0r6fvmt9Ovymat1WZ3b0oFsSoPyjWi19wusjLNm1+QDD+8b+ZfmZv/fd+7E0+nS+Lt+vFb0fja/7xP7kP+see/eN/soD99U8vrvM2X/zaXzw5/rVfzp/+1r/Rr7nza+3S/3Z/zd1fc+/XvPdr7v+a93/NT3/NB7/mwa/58Lf65Nf+jS6857f5NZm/MeLf4NdId1MIUFZmbVswTousSNtsRfRt8env/jy/AD0mk3R3+x43jrT6hf1WlzmRifAdAHZv+/4tgJlWMWCfuGbSiPRCPjCK0VDbGODf+jf4NXeYjPeJkJ8SKR8QCX+DgHy/LpPvt/41fosn/+hfPG22fy+alfUyXeQ0K7/xk4y03nVm/vpH/2qawkb++p2eVHVTzbaPJ9k/+vf89PZP5YtV+Y/+1frib/BkPctortvf9KSplhf1P/oXz+SL3+hZ/tP/6F9dyx+/3efX//ifXG9/UVGb7dfVqq709d/i29lPz/7Rv2f7STHPtO1v+O38Mteuf8fvEBf/YPsFaUpCdfv1D6pyWb1VmL9XtfhH/+K6WmyfEgny+qJayBe/yYt/9G/1EEleEnaK0+tqUZHS5T9+ZxKUSVUSIeiXFqC2n5ACuFAs3lBXSo4f+8lM8flNfzJvfrCq/9G/WrtKfopIL7/+1r+10j9gY2HvX2t379favfdr7e7/Wrv3f63dT3+t3Qe/1u7Br7X78Nfa26FJ+q19Pg8Z/sdoxmb8229of/uNvc+gm3+L1XpSFs38adbmxtD9gt8oW63Ka2v3WFuI7v/tn5OiI9mu2jYl/Ubs1VRvH5EA7Ij+/3W13a//JZT3o1Q+/TH99DfnT7O3xK7/6F+8yB6l+tZvfEvov3kU+m99A/RflxThbyIj+AW/4YvssrigsX5Vl7/ub0pGcZafZ+uyZS1LMH4DUjdsd9iWAe6v/+v+VtRMNHydtQVZS237O/CHP2hro5OnVhFbSvz6v+4voLd/f//1p9V0TSq/bSwcT/+mNHxi/LcViWX7j/6tM5gegfObMhy4D/bF3/U71QXxs7VTKSnR1vzVkrS/tbT/9X/d3wbvv6yLy2x6varKYnptwPwWx7OsvTTvzei9xUzZg977LfFesViRQWx+sF6Yl5Iz+5GdwF//1/2N0fjbebmyzV7nF6TIm3/0r76Yzf4f2Ro3yBcKAAA=",
                "__VIEWSTATE": "",
                "__EVENTVALIDATION": "/wEWBQL+raDpAgK3hdWgDwKYhJHtDwKSvba2CQKbgZmZA575eIrMtkZINDZao4TPeGOLl/xf",
                "userName": "Felhasználói+név",
                "password": "Jelszó",
                "ctl00$ContentPlaceHolder1$JobSearchForm1$txtKeyword": keyword,
                "ctl00$ContentPlaceHolder1$JobSearchForm1$btnSearch": "Keresés",
            },
        )
        return response.content
    except requests.exceptions.RequestException:
        print('HTTP Request failed')



Since we are expanding this scraper to make 2 separate searches, we're going to turn the CSV-writing code that we used before into a function so it can re-used at will:

In [11]:
import os
import csv

def write_or_append_to_csv(filename, headers, row):
    if not os.path.isfile(filename):
        with open(filename, 'w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(headers)
            writer.writerow(row)
    else: # else it exists so append without writing the header
        with open(filename, 'a', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(row)

The response is an HTML page which we'll have to parse. So we'll use BeautifulSoup to take the response string and parse the HTML:

Let's also add the date and time of retrieval. We'll put the date along with the count into an array so we can easily add it as a row to a CSV.

**NOTE** We are using timezone to set UTC as the time on everything, because that is what Github Actions uses.

In [12]:
from datetime import datetime, timezone
import time

current_date = datetime.now(timezone.utc).strftime("%m/%d/%Y")
current_time = datetime.now(timezone.utc).strftime("%H:%M:%S")

headers = ['date']
filename = 'vacancies_count.csv'

for keyword in keywords:
    headers.append(keyword)
    print(headers)

index = 2
row = [current_date]

for keyword in keywords:
    print(f"Scraping keyword: {keyword}")
    response = send_request(keyword)
    html = bs4.BeautifulSoup(response, 'html.parser')
    print(f"Returned HTML: {html.find(id='ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount')}")
    count = html.find(id='ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount').text
    row.append(count)
    time.sleep(2)

write_or_append_to_csv(filename, headers, row)

['date', 'gyógypedagógus']
['date', 'gyógypedagógus', 'tanár']
['date', 'gyógypedagógus', 'tanár', 'tanító']
['date', 'gyógypedagógus', 'tanár', 'tanító', 'óvodapedagógus']
['date', 'gyógypedagógus', 'tanár', 'tanító', 'óvodapedagógus', 'orvos']
['date', 'gyógypedagógus', 'tanár', 'tanító', 'óvodapedagógus', 'orvos', 'védőnő']
Scraping keyword: gyógypedagógus
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">656</span>
Scraping keyword: tanár
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">2898</span>
Scraping keyword: tanító
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">1370</span>
Scraping keyword: óvodapedagógus
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">1008</span>
Scraping keyword: orvos
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">213</span>
Scraping keyword: védőnő
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lbl

Now we'll send a second set of requests to get the number of teaching vacancies by *region*. To do this, we just have to send a POST request with a different set of data.

In [13]:
def send_regional_request(keyword, index):
    try:
        response = requests.post(
            url=url,
            params={
                "p": "1",
            },
            headers={
                "Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
                "Cookie": "ASP.NET_SessionId=nialgtjlnsftsb551ou1dj55",
            },
            data={
                "__EVENTTARGET": "",
                "__EVENTARGUMENT": "",
                "__VSTATE": "H4sIAAAAAAAEAO29B2AcSZYlJi9tynt/SvVK1+B0oQiAYBMk2JBAEOzBiM3mkuwdaUcjKasqgcplVmVdZhZAzO2dvPfee++999577733ujudTif33/8/XGZkAWz2zkrayZ4hgKrIHz9+fB8/Iv7vX/M3/e1+zV/3F/z+v/9JtWzrqmxe5b9oXdT5y6ppn2TTt79Xfv37//6/za/56/6W07bc2fnd3nz5cvd3m7TL59VFsfxNf91k+9N7ew8O6H/7v+nst/m1zvHPr/Vr0b+/3q/1a9oPzv2/f5tf5xf8WLFc5vW8XZS/7hePi8VF2tTTzz5a5LMiu1sssou8udvW2bJZZXW+bMYXxflH6VUxa+effbS7u/NROs+Li3lLf+zf+yidVPUsrz/7aOeju0e/4Nf/yaIpJmU+nzES3N3v8mt8w738Lr/mxezX+nUI/K8jY5Ku+JNfiz75bX6tX/BrU1e/7u86Ht8NemvyrJ7Of//Jum2rJfqb/Vq/tgcF//46578p3v913uTv2nyGQWAAv/av+1v/XkX5C/Nsum7zw5X5pfldfs35TGA4Ov9GwODX+QW/7rTMmubX/XUW+XJNBNh5nKXzOj//7KMVo/LtapGPs2b17qOjF9dF+wsrgZhW5SwrH9/NjgCWev51LIS9DoSTn9T3f+GX60V5uDRIlTkR9ad/IDB+XYHxGzV5mU9pQArro194rI3LX5jpWNK3eZ03dmizX+vXD/u/3+n/VX5RNC29M/tONWkUl5f/6F9cXv+jf/EPsjYrFoLCjxE9DZjZr/Ubh0A/7wB9WVfnRWko8ywv51nzg6XBsTRkKtLzwvz+1lDLTOFvQKNGn7/2r/sLfpLQaf/Rv7hJ84vr5gd5/Y//zTLKf/Svbup8PMMYpeWvtbc3A6r4i5iLheU3m/1mv/Vv9Gv8xgfpP/oHl21W/qN/8bJqfsfXP8jeLtbLtwT07T/6V69+8I//yWmxbP/Rv/oHefvb/17/6N/xA/rsvHr7j/49Kb6ljv56+q757fSbonlbldndi2KxJBoV68UvOF3kZZs2PyAYhFtDwP33fuxNPp0vi7frxW9HpGj+8T+5D/rHnv3jf7KA/fVPL67zNl/82l88Of61X86f/ta/0a+582vt0v92f83dX3Pv17z3a+7/mvd/zU9/zQe/5sGv+fC3+uTX/o0uvIeZGOP9DX6NdDf9R//qyzQrs7YtGKNFVqRttqKJaPHp7/48vwA1JpN0d/seN460+oX9Vpc5EYmwHQB2b/v+LYCZVjFgn7hm0uicuH5gFKOhtjHAv/Vv8GvuMBHvExk/JUI+IAL+Bh7xfl0m3m/9a/wWT/7Rv3jabP9eNCPrZbogrst/4ycZ6bjrzPz1j/7VNH2N/PU7PanqppptH0+yf/Tv+entn8oXq/If/av1xd/gyXqW0Ty3v+lJUy0v6n/0L57JF7/Rs/yn/9G/upY/frvPr//xP7ne/qKiNtuvq1Vd6eu/xbezn579o3/P9pNinmnb3/Db+WWuXf+O3yEO/sH2i+zimlDdfv2DqlxWbxXm71Ut/tG/uK4W26dEgLy+qBbyxW/y4h/9Wz1EkpeEneL0ulpUF9fyx+9MQjKpSiIE/dIC1PYTkroLxeINdaXk+LGfzBSf3/Qn8+YHq/of/au1q+SniPDy62/9Wyv1AxYW1v61dvd+rd17v9bu/q+1e//X2v3019p98GvtHvxauw9/rb0dmqLf2udxn9l/Q2j1X/BbrNaTsmjmT7M2N2brF/xG2WpVXlsr9tv8ZqQJREf8Ds9J35HcVm2bkl4h5mmqt4+IvXd3mAWk1R/ya34JFf4o3dq9k5J2a9rrMv/so2lVVvWj784L0lgficr7+C73PxWrxCrv91h9tvfx0R402td5997HR/dYG4pi+/UInR87LYk/8vTo6Bf8hi+yy+KCxvpVXf66v028cyg/owd/7V/3N+ehZG9JgP7Rv3iRPUrvzaDBb0OO3/z/XeT4LXvk+F1+/c00+K030eDXJTPzmzBj/DoE5zclP2OWn2frsmUg9NZvQGqZTTlrVmn2W1EzMZp11hbkgGjb34E//EFbGzM3tbZN+Qpv/wJ6+/f3X39aTddkRdvGwvHsVEozQ0ribUUKrP1H/1ZlCIPs7/8KjpV98Xf9TnVBsm9Nf0rGpjV/taQX3ypP4P3fBu+/rIvLbHq9qoh61wbMb3E8y9pL896M3lsYUaP3fku8VyxWZH2bH6wX5qXkzH6kvIXGvzEafzsvV7bZ6/yCDB4Z7ovZ7P8Bk1zkxjELAAA=",
                "__VIEWSTATE": "",
                "__EVENTVALIDATION": "/wEWOQL+raDpAgK3hdWgDwKYhJHtDwKSvba2CQKbgZmZAwLA6fftAQLf6bfuAQLf6bvuAQLf6fftAQLe6fftAQLd6fftAQLc6fftAQLb6fftAQLa6fftAQLZ6fftAQLI6fftAQLH6fftAQKR/K78CgKO/K78CgKP/K78CgKK/K78CgKM/K78CgKL/K78CgKN/K78CgKI/K78CgLtxZaVDALyxZaVDALzxZaVDALwxZaVDALxxZaVDAL2xZaVDAL3xZaVDAL0xZaVDALlxZaVDALqxZaVDALyxdaWDALyxdqWDALyxd6WDALyxeKWDALyxeaWDALyxeqWDALyxe6WDALyxfKWDALyxbaVDALyxbqVDALzxdaWDALtwth2ApWj/aoCAoqj/aoCAouj/aoCAvn49LQHAub49LQHAuf49LQHAqDT4tIEAr/T4tIEAr7T4tIEAr3T4tIEwBa5YVvDgBOk2zscOKtcboUTZeM=",
                "userName": "Felhasználói+név",
                "password": "Jelszó",
                "ctl00$ContentPlaceHolder1$JobSearchForm1$txtKeyword": keyword,
                "ctl00$ContentPlaceHolder1$JobSearchForm1$btnSearch": "Keresés",
                "ctl00$ContentPlaceHolder1$JobSearchForm1$ddlRequiredSchoolLevel":"0",
                "ctl00$ContentPlaceHolder1$JobSearchForm1$ddlRequiredExperience":"0",
                "ctl00$ContentPlaceHolder1$JobSearchForm1$ddlCounty": index,
                "ctl00$ContentPlaceHolder1$JobSearchForm1$txtCity":"",
                "ctl00$ContentPlaceHolder1$JobSearchForm1$ddlDurationOfLegalContract":"0",
                "ctl00$ContentPlaceHolder1$JobSearchForm1$ddlTypeOfEmploying":"0",
                "ctl00$ContentPlaceHolder1$JobSearchForm1$ddlJobType":"0"
            },
        )
        return response.content
    except requests.exceptions.RequestException:
        print('HTTP Request failed')

The regions are numbered from 1 to 20 in the request, so we'll send a total of 20 requests -- one for each region. As we did previously, we'll sleep for a couple of seconds so the server doesn't get mad at us. We'll map each number to the name of the region so that we have a pretty table.

In [15]:
regions = [
    {"id": "1",
    "name": "Bács-Kiskun megye"},
    {"id": "2",
    "name": "Baranya megye"},
    {"id": "3",
    "name": "Békés megye"},
    {"id": "4",
    "name": "Borsod-Abaúj-Zemplén megye"},
    {"id": "5",
    "name": "Budapest"},
    {"id": "6",
    "name": "Csongrád megye"},
    {"id": "7",
    "name": "Fejér megye"},
    {"id": "8",
    "name": "Győr-Moson-Sopron megye"},
    {"id": "9",
    "name": "Hajdú-Bihar megye"},
    {"id": "10",
    "name": "Heves megye"},
    {"id": "11",
    "name": "Jász-Nagykun-Szolnok megye"},
    {"id": "12",
    "name": "Komárom-Esztergom megye"},
    {"id": "13",
    "name": "Nógrád megye"},
    {"id": "14",
    "name": "Pest megye"},
    {"id": "15",
    "name": "Somogy megye"},
    {"id": "16",
    "name": "Szabolcs-Szatmár-Bereg megye"},
    {"id": "17",
    "name": "Tolna megye"},
    {"id": "18",
    "name": "Vas megye"},
    {"id": "19",
    "name": "Veszprém megye"},
    {"id": "20",
    "name": "Zala megye"}
]

filename = "teacher_jobs_by_region.csv"
headers = ["date"]
row = [current_date]

for region in regions:
    response = send_regional_request("tanár", region["id"])
    html = bs4.BeautifulSoup(response, 'html.parser')
    print(f"Returned HTML: {html.find(id='ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount')}")
    count = html.find(id='ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount').text
    headers.append(region["name"])
    row.append(count)
    time.sleep(3)

write_or_append_to_csv(filename, headers, row)

Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">90</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">36</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">48</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">297</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">538</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">63</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">220</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">152</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">95</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">63</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1_JobSearchForm1_lblCount">71</span>
Returned HTML: <span id="ctl00_ContentPlaceHolder1

Done! 👏